In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/Youga19/Tugas-Sistem-Rekomendasi/main/sistem%20rekomendasi.csv")
df.head()

,Mitra,Alamat,Modul
0,Setyo Hendro Prayoga,Puguh RT 2 RW 1 Kec.Pegandon Kab.Kendal,Implementasi Basis Data Web Service Sistem Bas...
1,Power Academy,Jl. Mampang Prapatan Raya no 84A Tegal Parang ...,Logika dan Konsep Teknologi AI Siklus Projek A...
2,Zenius,East Kuningan Setiabudi South Jakarta 12950 In...,Kemampuan Literasi Numerasi dan Berpikir Saint...
3,Hacktiv8,Gedung Aquarius Lt. 1&2 Jl. Sultan Iskandar Mu...,Memahami serta mengimplementasikan library hin...
4,Orbit Future Academy,Veteran RI Building 15th Floor Unit Z15-002 Pl...,Natural Language Processing Artificial Intelli...




1.   **Ikhtisar**



In [2]:
df.describe()

,Mitra,Alamat,Modul
count,9,9,9
unique,9,8,9
top,Setyo Hendro Prayoga,Menara Bidakara 1 Lantai 2 Jl. Gatot Subroto ...,Implementasi Basis Data Web Service Sistem Bas...
freq,1,2,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mitra   9 non-null      object
 1   Alamat  9 non-null      object
 2   Modul   9 non-null      object
dtypes: object(3)
memory usage: 344.0+ bytes



2.   **Tampilan Modul Pembelajaran (Sebelum Preprocessing)**



In [4]:
def print_description(index):
    example = df[df.index == index][['Modul', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [5]:
print_description(1)

Logika dan Konsep Teknologi AI Siklus Projek AI Pemrograman Python Metode Penelitian AI Proyek Akhir Kewirausahaan
Mitra: Power Academy 
Alamat: Jl. Mampang Prapatan Raya no 84A Tegal Parang Mampang Prapatan Jakarta Selatan


In [8]:
print_description(2)

Kemampuan Literasi Numerasi dan Berpikir Saintifik Dasar-Dasar Data Science Pythone Statistik Visualisasi Data dan Analisis Data
Mitra: Zenius 
Alamat: East Kuningan Setiabudi South Jakarta 12950 Indonesia


In [9]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining
Mitra: Setyo Hendro Prayoga
Alamat: Puguh RT 2 RW 1 Kec.Pegandon Kab.Kendal




3.   **Text Preprocessing**



In [10]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modul_pembelajaran_clean'] = df['Modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
df.head()

,Mitra,Alamat,Modul,modul_pembelajaran_clean
0,Setyo Hendro Prayoga,Puguh RT 2 RW 1 Kec.Pegandon Kab.Kendal,Implementasi Basis Data Web Service Sistem Bas...,implementasi basis data web service sistem bas...
1,Power Academy,Jl. Mampang Prapatan Raya no 84A Tegal Parang ...,Logika dan Konsep Teknologi AI Siklus Projek A...,logika dan konsep teknologi ai siklus projek a...
2,Zenius,East Kuningan Setiabudi South Jakarta 12950 In...,Kemampuan Literasi Numerasi dan Berpikir Saint...,kemampuan literasi numerasi dan berpikir saint...
3,Hacktiv8,Gedung Aquarius Lt. 1&2 Jl. Sultan Iskandar Mu...,Memahami serta mengimplementasikan library hin...,memahami serta mengimplementasikan library hin...
4,Orbit Future Academy,Veteran RI Building 15th Floor Unit Z15-002 Pl...,Natural Language Processing Artificial Intelli...,natural language processing artificial intelli...




4.   **Tampilan Modul Pembelajaran (Setelah Preprocessing)**


In [12]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modul_pembelajaran_clean', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [13]:
print_description_clean(1)

logika dan konsep teknologi ai siklus projek ai pemrograman python metode penelitian ai proyek akhir kewirausahaan
Mitra: Power Academy 
Alamat: Jl. Mampang Prapatan Raya no 84A Tegal Parang Mampang Prapatan Jakarta Selatan


In [14]:
print_description_clean(2)

kemampuan literasi numerasi dan berpikir saintifik dasardasar data science pythone statistik visualisasi data dan analisis data
Mitra: Zenius 
Alamat: East Kuningan Setiabudi South Jakarta 12950 Indonesia


In [15]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining
Mitra: Setyo Hendro Prayoga
Alamat: Puguh RT 2 RW 1 Kec.Pegandon Kab.Kendal




5.  **TF-IDF & Cosine Similarity**

In [16]:
df.set_index('Mitra', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modul_pembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.01959013, 0.06085598, 0.01327593, 0.02381961,
        0.07367311, 0.        , 0.        , 0.        ],
       [0.01959013, 1.        , 0.02243077, 0.        , 0.01742034,
        0.02451708, 0.        , 0.        , 0.01111824],
       [0.06085598, 0.02243077, 1.        , 0.        , 0.07698737,
        0.        , 0.        , 0.        , 0.02302558],
       [0.01327593, 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.01506932],
       [0.02381961, 0.01742034, 0.07698737, 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.07367311, 0.02451708, 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.43457932, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.43457932, 1.        , 0.        ],


In [17]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0     Setyo Hendro Prayoga
1           Power Academy 
2                  Zenius 
3                Hacktiv8 
4    Orbit Future Academy 
5             NF Computer 
6                Braindevs
7               Braindevs 
8           Binar Academy 
Name: Mitra, dtype: object



6. **Modelling**



In [18]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul



7. **Prediksi**



In [19]:
recommendations('Setyo Hendro Prayoga')

['NF Computer ', 'Zenius ', 'Orbit Future Academy ']

In [28]:
recommendations("Braindevs")

['Braindevs ', 'Setyo Hendro Prayoga', 'Power Academy ']